In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install -r requirements.txt


Explore stock market dataset from Yahoo Finance

In [ ]:
import yfinance as yf
import pandas as pd


## Prepare broad market indicies

In [ ]:
# Capture S&P500, NASDAQ100 and Russell 200 indecies and their equal weighted counter parts
# As well as VIX volatility index, DYX US Dollar index and TNX US 10 Year Treasuries Rate Index
broad_market_indicies = '^SPX ^SPXEW ^NDX ^NDXE ^RUT ^R2ESC ^VIX DX-Y.NYB ^TNX'

In [ ]:
broad_market = yf.download(broad_market_indicies, period='max', group_by='tickers') 
broad_market

In [ ]:
broad_market.to_csv('data/broad_market.csv.bz2', index='Date')

## Prepare Sector Indicies

In [ ]:
sector_indicies = 'XLE ^SP500-15 ^SP500-20 ^SP500-25 ^SP500-30 ^SP500-35 ^SP500-40 ^SP500-45 ^SP500-50 ^SP500-55 ^SP500-60'

In [ ]:
sectors = yf.download(sector_indicies, period='max') 
sectors

In [ ]:
sectors.to_csv('data/sectors.csv.bz2')

## Load list of IBD Growth Stocks

In [ ]:

ibd50 = pd.read_csv('data/IBD50.csv')
ibd50


In [ ]:
ibd250 = pd.read_csv('data/IBD250.csv')
ibd250

In [ ]:
# merge IBD 50 and 250 symbol sets
ibd50_set = set(ibd50['Symbol'])
ibd250_set = set(ibd250['Symbol'])
ibd_growth_set = ibd50_set.union(ibd250_set)

In [ ]:
len(ibd_growth_set)

In [ ]:
ibdgrowth_str = ' '.join(ibd_growth_set)
ibdgrowth_str



In [ ]:
stocks_ticker_set = ibd_growth_set

## Prepare stocks price data

In [ ]:
ibdgrowth_data = yf.download(ibdgrowth_str, period='max', group_by='tickers') 
ibdgrowth_data

In [ ]:
ibdgrowth_data.columns.levels

In [ ]:
ibdgrowth_data.to_csv('data/ibdgrowth_hist.csv.bz2', index='Date')

In [ ]:
ibdgrowth_loaded = pd.read_csv('data/ibdgrowth_hist.csv.bz2', header=[0, 1], index_col=0)
ibdgrowth_loaded

In [ ]:
for ticker in ibdgrowth_loaded.columns.levels[0][:2]:
    print(f'ticker: {ticker}')
    ticker_data = ibdgrowth_loaded[ticker]
    print(f'ticker historic data: {ticker_data}')
    # remove missing values
    ticker_data = ticker_data.dropna()
    print(f'ticker historic data without missing data: {ticker_data}')


## Prepare historical stock sales and earnings data

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

FMP_API_KEY=os.getenv("FMP_API_KEY")

print(f'FMP_API_KEY={FMP_API_KEY!= None}')

In [ ]:
import fmpsdk

# Company Valuation Methods
symbol: str = "AAPL"
symbols: ["AAPL", "CSCO", "QQQQ"]
exchange: str = "NYSE"
exchanges: ["NYSE", "NASDAQ"]
query: str = "AA"
limit: int = 3
period: str = "quarter"
download: bool = True
market_cap_more_than: int = 1000000000
beta_more_than: int = 1
volume_more_than: int = 10000
sector: str = "Technology"
dividend_more_than: int = 0
industry: str = "Software"
filing_type: str = "10-K"
print(f"Company Profile: {fmpsdk.company_profile(apikey=FMP_API_KEY, symbol=symbol)=}")


In [ ]:
# there should be no duplicate symbols in this list
assert not ibdgrowth_loaded.columns.levels[0].duplicated().any() 

In [ ]:
earnings_all_df = pd.DataFrame()
for ticker in stocks_ticker_set:
    earnings = fmpsdk.historical_earning_calendar(apikey=FMP_API_KEY, symbol=ticker, limit=-1)
    if earnings is not None and len(earnings) > 0:
        edf = pd.DataFrame(earnings)
        edf['date'] = pd.to_datetime(edf['date'])
        edf = edf.set_index(['symbol', 'date'])
        print(f"Earnings calendar for {ticker}: \n{edf}")
        # edf = edf.pivot(columns='symbol')
        # edf.swaplevel(i=0,j=1, axis=0)
        # edf.drop(columns=['symbol'])
        earnings_all_df = pd.concat([earnings_all_df, edf])
        print(f"Earnings calendar after pivot for {ticker}: \n{edf}")
        n_earnings = len(earnings)
        print(f"Total earnings reports for {ticker}: {n_earnings}")
#    earliest_earn = earnings[-1] if len(earnings > 0 else 'None')
#    print(f"Earliest earnings report for {ticker}: {earliest_earn}")


In [ ]:
earnings

In [ ]:
aaon = earnings_all_df.loc[['AAON']]

In [ ]:
aaon

In [ ]:
len(earnings_all_df)

In [ ]:
earnings_all_df


In [ ]:
earnings_file = 'data/earnings_calendar.csv.bz2'

In [ ]:
earnings_all_df.to_csv(earnings_file)

### Read back data and verify it

In [ ]:
import pandas as pd

earnings_loaded_df = pd.read_csv('data/earnings_calendar.csv.bz2', index_col=['symbol', 'date'])
print(earnings_loaded_df)

## Prepare key metrics data for company fundamentals

In [ ]:
keymetrics_all_df = pd.DataFrame()
for ticker in stocks_ticker_set:
    kms = fmpsdk.key_metrics(apikey=FMP_API_KEY, symbol=ticker, period='quarter', limit=-1)
    if kms is not None and len(kms) > 0:
        kms_df = pd.DataFrame(kms)
        kms_df['date'] = pd.to_datetime(kms_df['date'])
        kms_df = kms_df.set_index(['symbol', 'date'])
        print(f"Key metrics for {ticker} sample: \n{kms_df.columns}")
        keymetrics_all_df = pd.concat([keymetrics_all_df, kms_df])
        print(f"Key metrics concatenated {ticker}: \n{keymetrics_all_df.columns}")
        n_kms = len(kms_df)
        print(f"Total key metrics reports for {ticker}: {n_kms}")


Experiment with other stock data

In [ ]:
keymetrics_all_df

In [ ]:
kms_file = 'data/keymetrics_history.csv.bz2'
keymetrics_all_df.to_csv(kms_file)

## Prepare forward looking analyst estimates to be used as future covariates

In [ ]:
DEFAULT_LIMIT=-1
import typing
from fmpsdk.url_methods import __return_json_v3, __validate_period


def analyst_estimates(
    apikey: str, 
    symbol: str, 
    period: str = "annual",
    limit: int = DEFAULT_LIMIT
) -> typing.Optional[typing.List[typing.Dict]]:
    """
    Query FMP /analyst-estimates/ API.

    :param apikey: Your API key.
    :param symbol: Company ticker.
    :param period: 'annual' or 'quarter'
    :param limit: Number of rows to return.
    :return: A list of dictionaries.
    """
    path = f"/analyst-estimates/{symbol}"
    query_vars = {
        "apikey": apikey,
        "symbol": symbol,
        "period": __validate_period(value=period),
        "limit": limit,
    }
    return __return_json_v3(path=path, query_vars=query_vars)



In [ ]:

def fetch_estimates(period=None):
    assert period in ['quarter', 'annual']
    estimates_all_df = pd.DataFrame()
    for ticker in stocks_ticker_set:
        est = analyst_estimates(apikey=FMP_API_KEY, symbol=ticker, period=period, limit=-1)
        # print('est:', est)
        if est is not None and len(est) > 0:
            est_df = pd.DataFrame(est)
            est_df['date'] = pd.to_datetime(est_df['date'])
            est_df = est_df.set_index(['symbol', 'date'])
            # print(f"Analyst estimates for {ticker} sample: \n{est_df.columns}")
            estimates_all_df = pd.concat([estimates_all_df, est_df])
            # print(f"Key metrics concatenated {ticker}: \n{estimates_all_df.columns}")
            n_est = len(est_df)
            print(f"Total estimates reports for {ticker}: {n_est}")
    return estimates_all_df



In [ ]:
for p in ['annual', 'quarter']:
    estimates_all_df = fetch_estimates(p)
    est_file_name= f'data/analyst_estimates_{p}.csv.bz2'    
    estimates_all_df.to_csv(est_file_name)
    print(f'{p} estimates:', estimates_all_df)
    